In [11]:
import re
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [12]:
TOR_330_dem = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_dem.xlsx')

In [13]:
TOR_330_dem.groupby( ['Year','Status1'] )['Status1' ].count()

Year  Status1                     
2021  DNF                             286
      Finished                        431
2022  DNF                             363
      Finished at Bosses               88
      Finished at Courmayeur          408
      Finished at Rifugio Frassati    101
2023  DNF                             585
      Finished                        622
2024  DNF                             563
      Finished                        533
Name: Status1, dtype: int64

In [14]:
checkpoints_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_checkpoints_bibs.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_checkpoints_bibs.xlsx'

In [ ]:
all_aid_station_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_all_aid_station_bib_df.xlsx')

In [ ]:
lifebase_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_lifebase_bib.xlsx')

In [ ]:
# making duration hours 
datasets = [checkpoints_bib_df, lifebase_bib_df, all_aid_station_bib_df,  TOR_330_dem]
for df in datasets:
    df['Duration_hours'] = df['Duration_seconds']/ 3600 

### Getting Average and Median time for lifebases

In [ ]:
def creating_time_stats(df, column, category_order):
    df_merge = df.merge(
        TOR_330_dem[['PK', 'Finish Category']],
        on=['PK'],
        how='left')


    df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan

    stats_df = df_merge.groupby(['Finish Category', column])['Duration_seconds'].describe().reset_index(drop = False)


    stats_df[[ 'mean', 'std', 'min', '25%',
           '50%', '75%', 'max']] = stats_df[[ 'mean', 'std', 'min', '25%',
           '50%', '75%', 'max']].round(0)

    # Set 'Finish Category' as a categorical column with the defined order
    stats_df[column] = pd.Categorical(
        stats_df[column],
        categories=category_order,
        ordered=True)
    
    stats_df = stats_df.sort_values(by=column, ascending = True)
    
    

    for finish_category in df_merge['Finish Category'].unique():
        print(finish_category)
        stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'running_total_mean_seconds'
            ] =     stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'mean'
            ].cumsum()


        stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'running_total_median_seconds'
            ] =     stats_df.loc[
                (stats_df['Finish Category'] == finish_category), '50%'
            ].cumsum()


    stats_df = stats_df[['Finish Category', column,
            'count', 'mean', '50%', 'std', 'min', 'max', 
            'running_total_mean_seconds', 'running_total_median_seconds']]


    stats_df = stats_df.rename(columns={'count': f'Count_Finish_Category_{column}_seconds',
                                      'mean': f'Mean_Finish_Category_{column}_seconds',
                                      'std': f'STD_Finish_Category{column}t_seconds',
                                      '50%': f'Median_Finish_Category_{column}_seconds',
                                      'min': f'Min_Finish_Category_{column}_seconds', 
                                      'max': f'Max_Finish_Category_{column}_seconds'})
    
    stats_df.to_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_{column}_duration_stats_df.xlsx', index = False)

#     print(stats_df[stats_df['Finish Category'] == 'Sub-130'])
    return stats_df

In [ ]:
lifebase_stats_df = creating_time_stats(df = lifebase_bib_df, 
                    column = 'Lifebase', 
                    category_order = ['Start',
                            'Valgrisenche IN','Valgrisenche OUT',
                            'Cogne IN',  'Cogne OUT',
                            'Donnas IN', 'Donnas OUT', 
                            'Gressoney IN','Gressoney OUT',
                            'Valtournenche IN','Valtournenche OUT', 
                            'Ollomont IN','Ollomont OUT',
                            'FINISH'])

lifebase_stats_df

In [ ]:
checkpoints_stats_df = creating_time_stats(df = checkpoints_bib_df, 
                    column = 'Checkpoint', 
                    category_order = ['Start', 'La Thuile', 'Valgrisenche IN', 'Valgrisenche OUT',
                                       'Eaux Rousse', 'Cogne IN', 'Cogne OUT', 'Donnas IN', 'Donnas OUT',
                                       'Rifugio della Barma', 'Niel La Gruba', 'Gressoney IN',
                                       'Gressoney OUT', 'Champoluc', 'Valtournenche IN',
                                       'Valtournenche OUT', 'Oyace', 'Ollomont IN', 'Ollomont OUT',
                                       'FINISH'])

In [ ]:
all_aid_station_bib_stats_df = creating_time_stats(df = all_aid_station_bib_df, 
                    column = 'Aid Station', 
                    category_order = ['Start',
                                    'Baite Youlaz', 'La Thuile', 'Rifugio Deffeyes',
                                   'Planaval', 'Valgrisenche IN', 'Valgrisenche OUT', 'Chalet Epee',
                                   'Rhemes-Notre-Dame', 'Eaux Rousse', 'Rifugio Sella', 'Cogne IN',
                                   'Cogne OUT', 'Goilles', 'Rifugio Dondena', 'Chardonney', 'Pontboset',
                                   'Donnas IN', 'Donnas OUT', 'Perloz', 'Sassa', 'Rifugio Coda',
                                   'Rifugio della Barma', 'Lago Chiaro', 'Col della Vecchia',
                                   'Niel La Gruba', 'Loo', 'Gressoney IN', 'Gressoney OUT',
                                   'Rifugio Alpenzu', 'Champoluc', 'Rifugio Grand Tournalin',
                                   'Valtournenche IN', 'Valtournenche OUT', 'Rifugio Barmasse', 'Vareton',
                                   'Rifugio Magià', 'Rifugio Cuney', 'Bivacco R. Clermont', 'Oyace',
                                   'Bruson Arp', 'Ollomont IN', 'Ollomont OUT', 'Rifugio Champillon',
                                   'Ponteille Desot', 'Bosses', 'Rifugio Frassati', 'Pas Entre Deux Sauts',
                                   'Monte de la Saxe', 'FINISH'])

all_aid_station_bib_stats_df

### Who ran too easy or hard at the start?

In [ ]:
# sub_TOR_330_dem_bib_list = list(TOR_330_dem['PK'][TOR_330_dem['Finish Category'] == 'Sub-130'].unique())

# new_bib_list = []
# for bib in sub_TOR_330_dem_bib_list:
#     df = lifebase_bib_df[
#                 ((lifebase_bib_df['Duration_hours']> 14) &
#                 (lifebase_bib_df['Lifebase'] == 'Valgrisenche IN') )
#     &
#                 (lifebase_bib_df['PK']== bib)
#                ]
    
#     new_bib_list.append(df)
# df= pd.concat(new_bib_list)
# pk_unique = list(df['PK'].unique())

# for pk in pk_unique:
#     print(lifebase_bib_df[['PK','Lifebase', 'Timestamp', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '*'*40,  '\n',)
#     print(TOR_330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR_330_dem['PK'] == pk)  ], '\n','\n','\n','\n', '*'*40,  '\n',)

In [ ]:
# sub_TOR_330_dem_bib_list = list(TOR_330_dem['PK'][TOR_330_dem['Finish Category'] == 'Sub-140'].unique())

# new_bib_list = []
# for bib in sub_TOR_330_dem_bib_list:
#     df = lifebase_bib_df[
#                 ((lifebase_bib_df['Duration_hours']< 10) &
#                 (lifebase_bib_df['Lifebase'] == 'Valgrisenche IN') ) &
#                 (lifebase_bib_df['PK']== bib)]
#     new_bib_list.append(df)
    
# df= pd.concat(new_bib_list)

# pk_unique = list(df['PK'].unique())

# for pk in pk_unique:
#     print(lifebase_bib_df[['PK', 'Wave', 'Lifebase', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '\n',)
#     print(TOR_330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR_330_dem['PK'] == pk)  ], '\n', '*'*40,  '\n',)

In [ ]:
# # reading in Raw Data
# races = ['TOR330']
# years = [ 
# #     '2021',
# #         '2022',
# #          '2023', 
#     '2024'
#         ]

# TORX_df = {}

# for race in races:
#     for year in years:
#         df = pd.read_excel(f'{race} Data/1. 100x100trail/{race}_{year}.xlsx',
#                                  dtype={'Start Date': 'string',
#                                         'Year': 'string'})
#         print(f'{race}_{year} {df.shape}')
#         # Store the DataFrame in the dictionary with a key like 'TOR330_2021'
#         TORX_df[f'{race}_{year}'] = df
#     print('*'*50)
    
# TORX_df_concat = pd.concat(TORX_df)
# TOR330 = TORX_df_concat[TORX_df_concat['Year'] == year]

In [ ]:
# sub_TOR_330_dem_bib_list = list(TOR_330_dem['PK'][TOR_330_dem['Finish Category'] == 'Sub-90'].unique())

# new_bib_list = []
# for bib in sub_TOR_330_dem_bib_list:
#     df = lifebase_bib_df[
#                 ((lifebase_bib_df['Duration_hours']> 8) &
#                 (lifebase_bib_df['Lifebase'] == 'Valgrisenche OUT') ) &
#                 (lifebase_bib_df['PK']== bib)]
#     new_bib_list.append(df)
    
# df= pd.concat(new_bib_list)

# pk_unique = list(df['PK'].unique())

# for pk in pk_unique:
#     print(lifebase_bib_df[['PK', 'Wave', 'Lifebase', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '\n',)
#     print(TOR_330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR_330_dem['PK'] == pk)  ], '\n', '*'*40,  '\n',)